In [ ]:
1/ Bảng phân công công việc:
Tên thành viên	MSSV	Công việc	Tỉ lệ
Lê Tâm Anh	19127330	 Lấy thông tin Playlists bằng API	33%
Trần Minh Thiện	19127281	 Lấy thông tin Users bằng API	33%
Vương Thành An	19127326	 Lấy thông tin Tracks bằng API	34%

In [ ]:
import requests
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
import csv
import urllib3
import string
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không
import random


Hàm thu thập dữ liệu về user

In [ ]:
# lấy 1 lần 200 user bằng cách search chữ cái bắt đầu tên user 
#csv_filename là tên file user mình muốn tạo ra
#offset và limit sẽ được gắn vào url để lấy request
#offset là để biết mình muốn lấy từ vị trí nào trong danh sách các user đã tìm ra được
#limit là giới hạn số user trong 1 lần lấy (soundclould cho phép tối đa là 200) và mặc định là 50
def collect_users_api(csv_filename,offset,limit): 
          with open(csv_filename,"w") as f:
                csv_write=csv.writer(f)
                csv_write.writerow(["user_id","username","permalink","permalink_url","created_at"
                                              ,"last_modified","description","comments_count","followers_count","followings_count",
                                              "groups_count","likes_count","playlist_count","playlist_likes_count","track_count","reposts_count"
                                              ,"urn","verified","first_name","full_name","last_name","city","country_code","uri","avatar_url",
                                    "creator_subscriptions-product-id","creator_subscription-product-id",
                                    "visuals-urn","visuals-enabled","visuals-tracking","visuals-visuals-urn","visuals-visuals-entry_time",
                                    "visuals-visuals-visual_url","badges-pro","badges-pro_unlimited","badges-vertified",
                                    "station_urn","station_permalink"
                                            ])
 #open file và viết cột tên các thuộc tính của api user vào file csv               
                while True:
                      letter= random.choice(string.ascii_lowercase) # vòng lặp lấy ngẫu nhiên 1 chữ cái để search tên các user bắt đầu từ chữ cái đó
                      offset=offset+limit
                      if(offset>2000): # cho tới khi nào hơn 2000 user thì ngưng
                              break
                      while True:
                          r = requests.get(f'https://api-v2.soundcloud.com/search/users?q={letter}&client_id=qgbUmYdRbdAL2R1aLbVCgwzC7mvh8VKv&offset={offset}&limit={limit}')
                          data = json.loads(r.text)
                          #request bằng cách /search/users?q={chữ cái}  do nếu chỉ request 1 lần 1 user thì rất lâu nên khi search như vầy sẽ tiết kiệm rất nhiều thời gian      
                          # lấy data về và load thành dạng json
                          if(r.ok==True): # Kiểm tra điều kiện nếu request thành công thì sẽ lưu dữ liệu lấy được vào file csv
                                for item in data['collection']:
                                    if item['first_name']=="":
                                      item['first_name']='None'
                                    if item['description']=="":
                                      item['description']='None'
                                    if item['full_name']=="":
                                      item['full_name']='None'
                                    if item['last_name']=="":
                                      item['last_name']='None'
                                    if item['city']=="":
                                      item['city']='None'
                                    if item['country_code']=="":
                                      item['country_code']='None'
                                    if item['reposts_count']==None:
                                      item['reposts_count']=0
                                    if item['visuals']==None:          # Do dữ liệu trên api có cái có thuộc tính đó có cái không có nên phải if  để các thuộc tinh nào không có
                                                                       # thì cho lưu là None
                                                                       # Sau khi check hết các điều kiện thì lưu vào file  csv đúng theo thứ tự cột các thuộc tính đã viết vào phía trên       
                                      csv_write.writerow([item['id'],item['username'],item['permalink'],item['permalink_url'],item['created_at'],item['last_modified'],item['description'],item['comments_count']
                                                        ,item['followers_count'],item['followings_count'],item['groups_count'],item['likes_count']
                                                        ,item['playlist_count'],item['playlist_likes_count'],item['track_count'],item['reposts_count']
                                                        ,item['urn'],item['verified'],
                                                        item['first_name'],item['full_name'],item['last_name'],item['city'],item['country_code']
                                                        ,item['uri'],item['avatar_url'],
                                                        item['creator_subscriptions'][0]['product']['id'],
                                                        item['creator_subscription']['product']['id'],
                                                        'None','None','None',
                                                        'None',
                                                        'None',
                                                        'None',
                                                        item['badges']['pro'],
                                                        item['badges']['pro_unlimited'],
                                                        item['badges']['verified'],
                                                        item['station_urn'],
                                                        item['station_permalink']])
                                    else:
                                      if item['visuals']['tracking'] ==None:
                                        item['visuals']['tracking']='None'
                                      csv_write.writerow([item['id'],item['username'],item['permalink'],item['permalink_url'],
                                                          item['created_at'],item['last_modified'],item['description'],
                                                        item['comments_count']
                                                        ,item['followers_count'],item['followings_count'],item['groups_count'],item['likes_count']
                                                        ,item['playlist_count'],item['playlist_likes_count'],item['track_count'],item['reposts_count']
                                                        ,item['urn'],item['verified'],
                                                        item['first_name'],item['full_name'],item['last_name'],item['city'],item['country_code']
                                                        ,item['uri'],item['avatar_url'],
                                                        item['creator_subscriptions'][0]['product']['id'],
                                                        item['creator_subscription']['product']['id'],
                                                        item['visuals']['urn'],item['visuals']['enabled'],item['visuals']['tracking'],
                                                        item['visuals']['visuals'][0]['urn'],
                                                        item['visuals']['visuals'][0]['entry_time'],
                                                        item['visuals']['visuals'][0]['visual_url'],
                                                        item['badges']['pro'],
                                                        item['badges']['pro_unlimited'],
                                                        item['badges']['verified'],
                                                        item['station_urn'],
                                                        item['station_permalink']])
                                break
                          else:    # kiểm tra nếu request thất bại thì cho ngủ và request lại
                            time.sleep(1)
                     

In [ ]:
csv_file_user="user.csv"
offset=0
limit=200

In [ ]:
collect_users_api(csv_file_user,offset,limit) # gọi hàm để lấy data về file user.csv

Hàm thu thập dữ liệu về playlist của các user đã thu thập được

In [ ]:
def collect_playlist_api(csv_file_playlist,csv_file_user,limit):  # csv_file_user là tên file user để lấy dữ liệu
                                                                  #csv_file_playlist là tên file playlist 
                                                                  # offset cho bằng 0 vì nhóm em chọn lấy playlist của những user có từ 200 bài trở lên
                                                                  # do đó nếu offset quá lơn mà user chỉ có 200 bài thì sẽ bị lỗi
                                                                  # Nên để offset=0 để lấy 200 bài đầu tiên của mỗi user
          offset=0
          df = pd.read_csv(csv_file_user) 
          users_many_playlist=df.loc[df.playlist_count>=200]
          # load data từ file user thành dạng data frame của thư viện pandas sau đó dùng hàm loc để lấy ra những user có từ 200 playlist trở lên
          with open(csv_file_playlist,"w") as f:
                          csv_write=csv.writer(f)
                          csv_write.writerow(["playlist_id","title","permalink","permalink_url","created_at"
                                                        ,"last_modified","description","track_count","tracks_id","duration","embeddable_by",
                                                        "genre","likes_count","user_id","license","reposts_count","sharing","tag_list","label_name"
                                                        ,"managed_by_feeds","public","purchase_title","purchase_url","release_date",
                                              "secret_token","set_type","uri","artwork_url",
                                              "is_album","published_at","display_date"
                                                      ])
                          # tạo file playlist và viết trên các cột thuộc tính vào
                          # với từng user id ta sẽ ghép vào url như bên dưới /user_id/playlists... là sẽ lấy được thông tin những playlists của user đó
                          for user_id in users_many_playlist["user_id"]:
                                  offset=offset+200
                                  if offset >2000: # tương tự với user thì cũng lấy hơn 2000 playlist thì ngưng
                                    break
                                  while True:
                                    r = requests.get(f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id=qgbUmYdRbdAL2R1aLbVCgwzC7mvh8VKv&offset=0&limit={limit}')
                                    if r.ok==True:
                                        data = json.loads(r.text)
                                        for item in data['collection']:
                                                                  # chỗ này là lấy id của những track có trong playlist như thầy yêu cầu các id cách nhau bằng dấu - 
                                                                  # Nếu playlist không có track thì để None
                                              tracks_id=""
                                              for track in item['tracks']:
                                                tracks_id=tracks_id+str(track['id'])
                                                tracks_id+="-"
                                              tracks_id=tracks_id[:-1]
                                              if tracks_id=="":
                                                tracks_id='None'      
                                                # Do dữ liệu trên api có cái có thuộc tính đó có cái không có nên phải if  để các thuộc tinh nào không có
                                                      # thì cho lưu là None 
                                                      # Tương tự các thuộc tính bên dưới cũng được kiểm tra nếu không có thì để None                    
                                              if item['description']==None:
                                                item['description']='None'
                                              if item['label_name']==None:
                                                item['label_name']='None'
                                              if item['tag_list']=="":
                                                item['tag_list']='None'
                                              if item['managed_by_feeds']=="":
                                                item['managed_by_feeds']='None'
                                              if item['purchase_title']==None:
                                                item['purchase_title']='None'
                                              if item['purchase_url']==None:
                                                item['purchase_url']='None'
                                              if item['set_type']=="":
                                                item['set_type']='None'
                                              if item['genre']=="":
                                                item['genre']='None'
                                              if item['secret_token']==None:
                                                item['secret_token']='None'
                                              if item['release_date']==None:
                                                item['release_date']='None'
                                              # Sau khi check hết các điều kiện thì lưu vào file  csv đúng theo thứ tự cột các thuộc tính đã viết vào phía trên 
                                              csv_write.writerow([item['id'],item['title'],item['permalink'],item['permalink_url'],item['created_at']
                                                                          ,item['last_modified'],item['description'],item['track_count'],
                                                                  tracks_id,item['duration'],item['embeddable_by'],
                                                    item['genre'],item['likes_count'],item['user_id'],item['license'],item['reposts_count'],
                                                item['sharing'],item['tag_list'],item['label_name']
                                                    ,item['managed_by_feeds'],item['public'],item['purchase_title'],item['purchase_url'],item['release_date'],
                                                item['secret_token'],item['set_type'],item['uri'],item['artwork_url'],
                                                                item['is_album'],item['published_at'],item['display_date']
                                                                        ])
                                        break
                                    else: 
                                      time.sleep(1)
                    

In [ ]:
csv_file_playlist='playlist.csv'
limit=200
collect_playlist_api(csv_file_playlist,csv_file_user,limit) # gọi hàm để lấy dữ liệu về file playlist.csv

In [ ]:
# giống với playlist thì cũng sẽ lấy track của những user đã thu thập được ở file user.csv 
def collect_track_api(csv_file_track,csv_file_user,limit):
          offset=0
          df = pd.read_csv(csv_file_user) 
          users_many_tracks=df.loc[df.track_count>=200]            # hàm loc để lấy ra những user có số lượng track từ 200 trở lên
          with open(csv_file_track,"w") as f:
                          csv_write=csv.writer(f)
                          csv_write.writerow(["track_id","title","permalink","permalink_url","created_at"
                                                        ,"last_modified","description","duration","full_duration","embeddable_by",
                                                        "genre","likes_count","commentable","comment_count","downloadable","download_count"
                                                        ,"user_id","license","reposts_count","sharing","tag_list","label_name"
                                                        ,"public","publisher_metadata-id",
                                                        "publisher_metadata-urn","publisher_metadata-artist","publisher_metadata-album_title",
                                              "publisher_metadata-contains_music","publisher_metadata-writer_composer",
                                              "publisher_metadata-writer_isrc",
                                              "purchase_title","purchase_url","release_date",
                                              "secret_token","uri","artwork_url",
                                              "display_date","state","track_format","waveform_url",
                                              "media-first_transcoding-url","media-first_transcoding-preset","media-first_transcoding-duration",
                                              "media-first_transcoding-snipped","media-first_transcoding-format-protocol","media-first_transcoding-format-mime_type",
                                              "media-first_transcoding-quality","media-second_transcoding-url","media-second_transcoding-preset","media-second_transcoding-duration",
                                              "media-second_transcoding-snipped","media-second_transcoding-format-protocol","media-second_transcoding-format-mime_type",
                                              "media-second_transcoding-quality","media-third_transcoding-url","media-third_transcoding-preset","media-third_transcoding-duration",
                                              "media-third_transcoding-snipped","media-third_transcoding-format-protocol","media-third_transcoding-format-mime_type",
                                              "media-third_transcoding-quality","track_authorization","monetization_model","policy"
                                                      ])
                          for user_id in users_many_tracks["user_id"]:
                                  offset=offset+200
                                  if offset >=2500:
                                    break
                                  while True:
                                    r = requests.get(f'https://api-v2.soundcloud.com/users/{user_id}/tracks?client_id=qgbUmYdRbdAL2R1aLbVCgwzC7mvh8VKv&offset=0&limit={limit}')
                                    # với từng user id ta sẽ ghép vào url như bên dưới /user_id/tracks... là sẽ lấy được thông tin những tracks của user đó
                                    if r.ok==True:
                                        data = json.loads(r.text)
                                        for item in data['collection']:
                                          #ở đây vì các thuộc tính của track loại có loại không có cho nên em quan sát nhiều user và trích ra được tất cả những thuộc tính
                                          # có thể có của 1 track sau đó lưu vào 1 temp dictionary với tất cả thuộc tính đều có giá trị là None
                                          # Sau đó sẽ cho chạy và lưu từng thuộc tính có của track đó có vào đây và những thuộc tính tồn tại thì nó sẽ thay đổi tempdata
                                          # và những thuộc tính mà track đó không có thì sẽ giữ nguyên là None
                                          # ở thuộc tính media transcodings có khoảng tối đa 3 loại nên em sẽ lưu là first second third
                                              tempdata={
                                                      "artwork_url": 'None',
                                                      "caption": 'None',
                                                      "commentable": 'None',
                                                      "comment_count": 'None',
                                                      "created_at": 'None',
                                                      "description": 'None',
                                                      "downloadable": 'None',
                                                      "download_count":'None',
                                                      "duration": 'None',
                                                      "full_duration": 'None',
                                                      "embeddable_by": 'None',
                                                      "genre": 'None',
                                                      "has_downloads_left": 'None',
                                                      "id":'None',
                                                      "kind": 'None',
                                                      "label_name": 'None',
                                                      "last_modified": 'None',
                                                      "license": 'None',
                                                      "likes_count": 'None',
                                                      "permalink": 'None',
                                                      "permalink_url": 'None',
                                                      "playback_count": 'None',
                                                      "public": 'None',
                                                      "publisher_metadata": {
                                                                  "id": 'None',
                                                                  "urn": 'None',
                                                                  "artist": 'None',
                                                                  "album_title":'None',
                                                                  "contains_music": 'None',
                                                                  "writer_composer":'None',
                                                                  "isrc": 'None',
                                                                  },
                                                      "purchase_title":'None',
                                                      "purchase_url":'None',
                                                      "release_date": 'None',
                                                      "reposts_count": 'None',
                                                      "secret_token": 'None',
                                                      "sharing": 'None',
                                                      "state": 'None',
                                                      "streamable": 'None',
                                                      "tag_list": 'None',
                                                      "title": 'None',
                                                      "track_format": 'None',
                                                      "uri": 'None',
                                                      "urn": 'None',
                                                      "user_id": 'None',
                                                      "visuals": 'None',
                                                      "waveform_url": 'None',
                                                      "display_date": 'None',
                                                      "media": {
                                                      "transcodings": [
                                                      {
                                                      "url": 'None',
                                                      "preset": 'None',
                                                      "duration": 'None',
                                                      "snipped": 'None',
                                                      "format": {
                                                      "protocol": 'None',
                                                      "mime_type": 'None'
                                                      },
                                                      "quality": 'None'
                                                      },
                                                      {
                                                      "url": 'None',
                                                      "preset": 'None',
                                                      "duration": 'None',
                                                      "snipped": 'None',
                                                      "format": {
                                                      "protocol": 'None',
                                                      "mime_type": 'None'
                                                      },
                                                      "quality": 'None'
                                                      },{
                                                        "url": 'None',
                                                        "preset": 'None',
                                                        "duration": 'None',
                                                        "snipped": 'None',
                                                        "format": {
                                                        "protocol": 'None',
                                                        "mime_type": 'None'
                                                        },
                                                        "quality": 'None'
                                                        }
                                                      ]
                                                      },
                                                      "station_urn": 'None',
                                                      "station_permalink": 'None',
                                                      "track_authorization": 'None',
                                                      "monetization_model": 'None',
                                                      "policy": 'None',
                                                      }
                                              # Điền các thuộc tính lấy được vào tempdata
                                              for key in list(item.keys()):
                                                  if key=='publisher_metadata':
                                                    if item['publisher_metadata']!=None:
                                                      for pm_key in list(item['publisher_metadata'].keys()):
                                                        if item['publisher_metadata'][pm_key]!=None or item['publisher_metadata'][pm_key]!="":
                                                          tempdata['publisher_metadata'][pm_key]=item['publisher_metadata'][pm_key]
                                                  elif key=='media':
                                                      if len(item['media']['transcodings'])>10:
                                                        i=0
                                                        for m_t_item in item['media']['transcodings']:
                                                          for m_t_key in list(item['media']['transcodings'][i].keys()):
                                                            tempdata['media']['transcodings'][i][m_t_key]=m_t_item[m_t_key]
                                                          i=i+1
                                                  else:
                                                      if item[key]== None:
                                                        tempdata[key]='None'
                                                      elif item[key]== "":
                                                        tempdata[key]='None'
                                                      else:
                                                        tempdata[key]=item[key]
                                              # Viết thông tin tracks vào file csv        
                                              csv_write.writerow([tempdata['id'],tempdata['title'],tempdata['permalink'],tempdata['permalink_url'],tempdata['created_at']
                                                              ,tempdata['last_modified'],tempdata['description'],tempdata['duration'],tempdata['full_duration'],tempdata['embeddable_by'],
                                                              tempdata['genre'],tempdata['likes_count'],tempdata['commentable'],tempdata['comment_count'],tempdata['downloadable'],tempdata['download_count']
                                                              ,tempdata['user_id'],tempdata['license'],tempdata['reposts_count'],
                                                              tempdata['sharing'],tempdata['tag_list'],tempdata['label_name']
                                                              ,tempdata['public'],
                                                              tempdata['publisher_metadata']['id'],
                                                              tempdata['publisher_metadata']['urn'],tempdata['publisher_metadata']['artist'],
                                                              tempdata['publisher_metadata']['album_title'],
                                                              tempdata['publisher_metadata']['contains_music'],
                                                              tempdata['publisher_metadata']['writer_composer'],
                                                              tempdata['publisher_metadata']['isrc']
                                                    ,tempdata['purchase_title'],tempdata['purchase_url'],tempdata['release_date'],
                                                    tempdata['secret_token'],tempdata['uri'],tempdata['artwork_url'],
                                                    tempdata['display_date'],tempdata['state'],tempdata['track_format'],tempdata['waveform_url'],
                                                    tempdata['media']['transcodings'][0]['url'],
                                                    tempdata['media']['transcodings'][0]['preset'],
                                                    tempdata['media']['transcodings'][0]['duration'],
                                                    tempdata['media']['transcodings'][0]['snipped'],
                                                      tempdata['media']['transcodings'][0]['format']['protocol'],
                                                      tempdata['media']['transcodings'][0]['format']['mime_type'],tempdata['media']['transcodings'][0]['quality'],
                                                          tempdata['media']['transcodings'][1]['url'],
                                                    tempdata['media']['transcodings'][1]['preset'],
                                                    tempdata['media']['transcodings'][1]['duration'],
                                                    tempdata['media']['transcodings'][1]['snipped'],
                                                      tempdata['media']['transcodings'][1]['format']['protocol'],
                                                      tempdata['media']['transcodings'][1]['format']['mime_type'],tempdata['media']['transcodings'][1]['quality']
                                                      ,tempdata['media']['transcodings'][2]['url'],
                                                    tempdata['media']['transcodings'][2]['preset'],
                                                    tempdata['media']['transcodings'][2]['duration'],
                                                    tempdata['media']['transcodings'][2]['snipped'],
                                                      tempdata['media']['transcodings'][2]['format']['protocol'],
                                                      tempdata['media']['transcodings'][2]['format']['mime_type'],tempdata['media']['transcodings'][2]['quality'],
                                                                      tempdata['track_authorization'],tempdata['monetization_model'],tempdata['policy']
                                                            ])

                                        break
                                    else:
                                      time.sleep(1)
          

In [ ]:
# Gọi hàm lấy dữ liệu tracks
csv_file_track='track.csv'
limit=200
collect_track_api(csv_file_track,csv_file_user,limit)

In [18]:
import pandas as pd
df = pd.read_csv('user.csv',error_bad_lines=False) 
print(df.mean())
print(df.median())

user_id                 1.898707e+08
comments_count          1.935090e+02
followers_count         3.426145e+03
followings_count        1.701055e+02
groups_count            0.000000e+00
likes_count             3.484605e+02
playlist_count          2.372750e+01
playlist_likes_count    2.116450e+01
track_count             1.236640e+02
reposts_count           0.000000e+00
verified                5.450000e-02
badges-pro              0.000000e+00
badges-pro_unlimited    0.000000e+00
badges-vertified        5.450000e-02
dtype: float64
user_id                 87271900.5
comments_count                 2.0
followers_count             1006.5
followings_count               7.0
groups_count                   0.0
likes_count                   10.0
playlist_count                 3.0
playlist_likes_count           0.0
track_count                   45.0
reposts_count                  0.0
verified                       0.0
badges-pro                     0.0
badges-pro_unlimited           0.0
badges-verti

In [30]:
import pandas as pd
df = pd.read_csv('user.csv',error_bad_lines=False) 

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   user_id                           2000 non-null   int64 
 1   username                          1999 non-null   object
 2   permalink                         2000 non-null   object
 3   permalink_url                     2000 non-null   object
 4   created_at                        2000 non-null   object
 5   last_modified                     2000 non-null   object
 6   description                       1365 non-null   object
 7   comments_count                    2000 non-null   int64 
 8   followers_count                   2000 non-null   int64 
 9   followings_count                  2000 non-null   int64 
 10  groups_count                      2000 non-null   int64 
 11  likes_count                       2000 non-null   int64 
 12  playlist_count      

In [31]:
import pandas as pd
df = pd.read_csv('track.csv') 
print(df.mean())
print(df.median())
print(df.info())

track_id          8.562991e+08
duration          1.226630e+06
full_duration     1.226630e+06
likes_count       8.332834e+00
commentable       9.006986e-01
downloadable      2.120758e-01
download_count    1.275449e+00
user_id           2.057640e+08
reposts_count     1.034930e+00
public            1.000000e+00
dtype: float64
track_id          935782904.5
duration             715977.0
full_duration        715977.0
likes_count               1.0
commentable               1.0
downloadable              0.0
download_count            0.0
user_id           142679120.0
reposts_count             0.0
public                    1.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004 entries, 0 to 2003
Data columns (total 64 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   track_id                                   2004 non-null   int64 
 1   title                              

In [24]:
import pandas as pd
df = pd.read_csv('track.csv') 
print(df.reposts_count.mean())

1.0349301397205588


In [32]:
import pandas as pd
df = pd.read_csv('playlist.csv') 
print(df.mean())
print(df.median())
print(df.info())

playlist_id         4.475962e+08
track_count         1.202825e+00
duration            8.116483e+05
likes_count         1.158930e+00
user_id             1.868966e+08
reposts_count       0.000000e+00
managed_by_feeds    0.000000e+00
public              1.000000e+00
is_album            1.967709e-02
dtype: float64
playlist_id         584952316.5
track_count                 0.0
duration               232798.0
likes_count                 0.0
user_id              95871939.0
reposts_count               0.0
managed_by_feeds            0.0
public                      1.0
is_album                    0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   playlist_id       1982 non-null   int64 
 1   title             1982 non-null   object
 2   permalink         1982 non-null   object
 3   permalink_url     1982 non-null   object
 4

In [28]:
import pandas as pd
df = pd.read_csv('playlist.csv') 
print(df.reposts_count.mean())

0.0
